In [274]:
from nltk import word_tokenize
from nltk.corpus import stopwords, words
from nltk.corpus import wordnet as wn
from  spellchecker import SpellChecker
import wordninja
import typo
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sys import platform, path
import json
import csv
from time import time

In [275]:
win_path = r"C:\Users\Anton\Desktop\projects\spell_checker\csv_imports\\"
mac_linux_path = "csv_imports/"

In [276]:
if platform == "linux" or platform == "linux2":
    path = mac_linux_path
elif platform == "darwin":
    path = mac_linux_path
elif platform == "win32":
    path = win_path
else:
    path = win_path

In [277]:
porn_terms_df = pd.read_csv(path + "porn_words.csv")
porn_words = porn_terms_df["term"].unique()
pornwords_set_lower = {w.lower() for w in porn_words}

In [278]:
queries_df = pd.read_csv(path + "nlp_queries.csv")
queries_df = queries_df.drop(columns=["#"])
queries_df = queries_df.fillna("")

#### Get a list of all single words in the query-column
#### Also retrieve stats about the data

In [279]:
single_words_set = set()
number_of_nonunique_words = 0
for q in queries_df["query"].unique():
    number_of_nonunique_words += len(q.split())
    single_words_set.update(set(q.split()))


In [280]:
print("Number of queries:", f"{len(queries_df):,}")
print("Number of non-unique words:", f"{number_of_nonunique_words:,}")
print("Number of unique words:", f"{len(single_words_set):,}")

Number of queries: 1,062,097
Number of non-unique words: 3,147,323
Number of unique words: 202,507


In [281]:
# japanese <--- {japonese, japanaise, jpanese, japanes}

Remove stopwords from queries

In [282]:
stop_words = set(stopwords.words('english')) 

In [283]:
def remove_stop_words(sentence): 
    words = sentence.split() 
    filtered_words = [word for word in words if word not in stop_words] 

    return ' '.join(filtered_words)

In [284]:
spell = SpellChecker()

#### Feed the dictionary specific porn-terms
* Pornstars names
* Studios names
* List of movies and series (Focus on games and animes)
* Akronyms
* proper names

Load Pornstar names

Studios names

Load akronyms

In [285]:
with open('/Users/antongigele/Desktop/python/spell_checker/csv_imports/porn_acronyms.json') as json_file:
    acronyms_dict = json.load(json_file)

Proper names aka porn-neologisms

In [286]:
proper_porn_words = ['hentai', 'omegle', 'sexmex', 'roblox', 'pinay', 'futanari']

Load into dictionary

In [287]:
spell.word_frequency.load_words(list(acronyms_dict.keys()))
# spell.word_frequency.load_words(proper_porn_words)

#### Ideas
* Use wordnet (synset: all the meanings a single word can have)

In [288]:
for word in wn.synsets('hello'):
    print(word, word.definition())

Synset('hello.n.01') an expression of greeting


In [289]:
def spellcheck_porn(query):
    # start = time()
    query = remove_stop_words(query)
    query_list = word_tokenize(query)
    mispelled = spell.unknown(query_list)
    if len(mispelled) == 0:
        # end = time()
        # print(end-start)
        return query_list
    else:
        result_list = [w for w in query_list if w not in mispelled]
        for word in mispelled:
            # candidates = spell.candidates(word)
            if spell.candidates(word) == None:
                word = " ".join(wordninja.split(word))
                result_list += spellcheck_porn(word)
            elif len(list(set(spell.candidates(word)).intersection(pornwords_set_lower))) == 0:
                result_list.append(spell.correction(word))
            else:
                result_list.append(list(set(spell.candidates(word)).intersection(pornwords_set_lower))[0])
        # end = time()
        # print(end-start)
        return result_list

In [290]:
split_words = wordninja.split("masrerbation")
split_words_str = " ".join(split_words)

In [291]:
split_words_str

'mas rer bati on'

In [292]:
word_tokenize(split_words_str)

['mas', 'rer', 'bati', 'on']

In [295]:
spellcheck_porn("masturbataon")

['masturbation']

In [294]:
var = list(single_words_set)[:100]

In [273]:
strange_words = []
known_words = []
for word in list(single_words_set)[:100]:
    try:
        known_words.append(spellcheck_porn(word))
    except TypeError:
        strange_words.append(word)

KeyboardInterrupt: 

In [135]:
strange_words

['virgensitas',
 'visconde',
 'schulmaedchen',
 'pornzoo3dblogspotcom',
 'httpsnw',
 'sapnabhabhi',
 'remerciement',
 'stronggirl',
 'australianhomemade',
 'jwtiescuffed',
 'lyubov',
 'kv191',
 'abp933',
 'brownsuga',
 'eslammustafa1the',
 'bustybabydolls',
 'superdotados',
 'athnessa',
 'alicekosmos',
 'alberona',
 'brutalmaster',
 'jessitron',
 'redpaganmom',
 'bathroomgay',
 '2girlssametime',
 'wev0011',
 'wifecrazystacey',
 'wienerfuck',
 'savannabond',
 'filipinasex',
 'schenectady',
 'montpellier',
 'seasonless']

In [78]:
for word in spellcheck_porn("fold"):
    print(word)

NameError: name 'time' is not defined

#### Detect false-correction
bqc ------> bwc, but got corrected to "bbc" because of the higher frequency of "bbc"

In [35]:
for word in spellcheck_porn("bqc"):
    print(word)

{'bic', 'bbc', 'bac', 'bwc', 'bec'}
bbc


example of queries belonging to the same cluster

In [16]:
query_1 = "japanese milf with big tit"
query_2 = "japanese milf big tits"

In [17]:
spellcheck_porn(query_1)

'japanese milf big tit'

#### Apply to all queries

In [18]:
queries_df["shorted_corrected_query"] = queries_df["query"].apply(spellcheck_porn)

{'pinny', 'piney', 'pina', 'pinky'}
{'dacha'}
{'compo', 'pompom', 'compos', 'momo'}
None


TypeError: 'NoneType' object is not iterable